In [ ]:
#!pip install tensorflow==2.7
#!pip install transformers
#!pip install gpustat datasets
#from google.colab import drive
from transformers import pipeline
from datasets import load_dataset
import logging
import random

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 68.7 MB/s 
     |████████████████████████████████| 67 kB 8.1 MB/s 
     |████████████████████████████████| 6.8 MB 30.3 MB/s 
     |████████████████████████████████| 895 kB 62.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 312 kB 8.5 MB/s 
     |████████████████████████████████| 133 kB 9.6 MB/s 
     |████████████████████████████████| 1.1 MB 7.5 MB/s 
     |████████████████████████████████| 243 kB 20.5 MB/s 
     |████████████████████████████████| 271 kB 22.2 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 144 kB 22.3 MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=345ba4d33439e18412b4

In [ ]:
import os
import numpy as np
import torch
dataset = load_dataset("aeslc")
logging.basicConfig(
    format="%(asctime)s — %(levelname)s — %(name)s — %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO
)
logger = logging.getLogger()

def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


MODEL = "gpt2-medium"
# MODEL = "distilgpt2"
# MODEL = "sshleifer/tiny-gpt2"
SPECIAL_TOKENS = {
    "sep_token": "<|SEP|>",
}
MAXLEN = 768
TRAIN_SIZE = 0.9
EPOCHS = 4
LR = 5e-4
EPS = 1e-8
WARMUP_PROPORTION = 0.1
UNFREEZE_LAST_N = 6

SEED = 54321

TRAIN_BATCHSIZE = 1
# fp16 only useful in batch_size > 8
seed_everything(SEED)

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/159M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/572k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [1]:
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForPreTraining,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer.add_special_tokens(SPECIAL_TOKENS)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

config = AutoConfig.from_pretrained(
    MODEL,
    sep_token_id=tokenizer.sep_token_id,
    output_hidden_states=False,
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForPreTraining.from_pretrained(MODEL,output_hidden_states=False, config=config).to(device)
# Special tokens added, model needs to be resized accordingly
model.resize_token_embeddings(len(tokenizer))
def encode(examples):
  return tokenizer(examples['text'], truncation=True, max_length=MAXLEN, padding="max_length")
text_dataset = dataset.map(lambda x:{"text": x["subject_line"] + SPECIAL_TOKENS['sep_token'] + x["email_body"]})

NameError: name 'MODEL' is not defined

In [ ]:
train = text_dataset['train']['text'] + text_dataset['validation']['text']
test = text_dataset['test']['text']

In [ ]:
from torch.utils.data import Dataset
class EmailDataset(Dataset):
    def __init__(self, data, tokenizer, randomize=True):

        self.randomize = randomize
        self.tokenizer = tokenizer
        self.data = data

    # ---------------------------------------------#

    def __len__(self):
        return len(self.data)

    # ---------------------------------------------#

    def __getitem__(self, i):
        text = self.data[i]

        encodings_dict = self.tokenizer(text, truncation=True, max_length=MAXLEN, padding="max_length")

        input_ids = encodings_dict["input_ids"]
        attention_mask = encodings_dict["attention_mask"]

        return {
            "label": torch.tensor(input_ids),
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
        }


In [ ]:
train_dataset = EmailDataset(train, tokenizer)
val_dataset = EmailDataset(test, tokenizer, randomize=False)

In [ ]:
TRAINING_SET_SIZE = len(train_dataset)
warmup_steps = (TRAINING_SET_SIZE / TRAIN_BATCHSIZE) * WARMUP_PROPORTION * EPOCHS

In [ ]:
from datasets.utils import logging
from transformers.utils import logging

for parameter in model.parameters():
    parameter.requires_grad = False

for i, m in enumerate(model.transformer.h):
    # Only un-freeze the last n transformer blocks
    if i + 1 > 12 - UNFREEZE_LAST_N:
        for parameter in m.parameters():
            parameter.requires_grad = True

for parameter in model.transformer.ln_f.parameters():
    parameter.requires_grad = True

for parameter in model.lm_head.parameters():
    parameter.requires_grad = True


training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/code/emailwriter/models/aelsc",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=TRAIN_BATCHSIZE,
    per_device_eval_batch_size=TRAIN_BATCHSIZE,
    eval_accumulation_steps=10,
    save_steps=int(TRAINING_SET_SIZE / 2),
    evaluation_strategy="steps",
    eval_steps=int(TRAINING_SET_SIZE / 2),
    warmup_steps=warmup_steps,
    learning_rate=LR,
    adam_epsilon=EPS,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    dataloader_num_workers=2,
)
# --
# set the main code and the modules it uses to the same log-level according to the node
log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
logging.set_verbosity(log_level)
logging.set_verbosity(log_level)
# ---------------------------------------------------#
trainer = Seq2SeqTrainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset, tokenizer=tokenizer
)

# ---------------------------------------------------#
trainer.train()
trainer.save_model()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 16396
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 65584


Step,Training Loss,Validation Loss
8198,0.798800,0.757307
16396,0.742300,0.707871
24594,0.646000,0.677492
32792,0.665400,0.648508
40990,0.442700,0.655080
49188,0.469800,0.639641
57386,0.261100,0.716663
65584,0.272800,0.717468


***** Running Evaluation *****
  Num examples = 1906
  Batch size = 1
Saving model checkpoint to /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-8198
Configuration saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-8198/config.json
Model weights saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-8198/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-8198/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-8198/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1906
  Batch size = 1
Saving model checkpoint to /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-16396
Configuration saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-16396/config.json
Model weights saved in /content/drive/MyDrive/code/emailwriter/models/aelsc/checkpoint-16396/pytorch_mod